In [1]:
#I want to avoid using Pandas until the end
#I want the final DataFrame to have rows of this
#Event Num, team id, player id, game seconds left, shot clock seconds, LOC_X,LOC_Y,LOC_Z

In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
def json2csv(game_path):

    #LOAD JSON DATA
    data = json.load(open(game_path))

    game_id = data['gameid']

    full_df=[]

    for event in range(len(data['events'])):

        moments = data['events'][event]['moments']

        df = np.array([np.array(moment[5]) for moment in moments])
        print(df)

        #FIX INCORRECTLY SIZED ARRAYS

        #Find all the indices where the length of the tracking data matrix isn't 11
        short_indices = [(np.shape(df[i])[0],i) for i,d in enumerate(moments) if len(d[5])!=11]
        #Keep all the ones where there's still 9 in there - we don't want any where the players aren't there
        df = np.array([x for x in df if len(df)>9])

        #Tests if array is incorrectly sized
        if(len(short_indices)!=0):
            #For all short indices in short indices array
            for short_index in short_indices:
                num_inserts = 11-short_index[0]
                #Perform insert certain number of times depending on 
                for num_insert in range(num_inserts):
                    df[short_index[1]]=np.insert(df[short_index[1]],0,np.array([-100.0,-100.0,0.0,0.0,0.0]),axis=0)

            #Turn from 3d array to 2d
            df = np.vstack(df)
            
            #To 3d array for consistency with others
            df=np.reshape(df,newshape=(int(df.shape[0]/11),11,5))
        
        if(len(df.shape)==3):
            df=np.reshape(df,newshape=(df.shape[0]*df.shape[1],5))

            event_col = np.full(shape=(len(df),1),fill_value=event+1)
            df = np.hstack((event_col,df))
            full_df.append(df)

    #Regular python list to array
    full_df=np.array(full_df)
    #Stack all the columns into one big 2d array
    full_df = np.vstack(full_df)

    #Convert to DataFrame
    full_df=pd.DataFrame(full_df,columns=['GAME','EVENT','TEAM_ID','PLAYER_ID','LOC_X','LOC_Y','LOC_Z'])

    return full_df, game_id

def save_DataFrame(DataFrame,file_path):
    pd.DataFrame.to_csv(DataFrame,file_path)

In [54]:
data = json.load(open('../data/json/jan.json'))

In [87]:
def shorter_json2csv(game_path):

    #LOAD JSON DATA
    data = json.load(open(game_path))

    game_id = data['gameid']

    full_df=[]

    for event in range(len(data['events'])):

        moments = data['events'][event]['moments']

        lengths = np.array([len(x[5]) for x in moments])

        if len(lengths)==0:
            continue
        if np.all(lengths == lengths[0])==False:
            continue

        game_clock = np.array([moment[2] for moment in moments])
        shot_clock = np.array([moment[3] for moment in moments])

        motion = np.array([np.array(moment[5]) for moment in moments])

        motion=np.reshape(motion,newshape=(motion.shape[0]*motion.shape[1],5))

        event_col = np.full(shape=(len(motion),1),fill_value=(event+1))
        # game_clock_arr = np.full(shape=(len(motion),1),fill_value=(game_clock))
        game_clock = np.reshape(np.repeat(game_clock,11),newshape=(-1,1))
        # shot_clock_arr = np.full(shape=(len(motion),1),fill_value=(shot_clock))
        shot_clock = np.reshape(np.repeat(shot_clock,11),newshape=(-1,1))

        df = np.hstack((event_col,motion,game_clock,shot_clock))
        
        full_df.append(df)

    #Regular python list to array
    full_df=np.array(full_df)
    #Stack all the columns into one big 2d array
    full_df = np.vstack(full_df)

    #Convert to DataFrame
    full_df=pd.DataFrame(full_df,columns=['EVENT','TEAM_ID','PLAYER_ID','LOC_X','LOC_Y','LOC_Z','GAME_CLOCK','SHOT_CLOCK'])

    return full_df, game_id

In [88]:
df,id=shorter_json2csv('../data/json/jan.json')

(1650, 1) (1650, 5) (1650, 1) (1650, 1)
(12650, 1) (12650, 5) (12650, 1) (12650, 1)
(12650, 1) (12650, 5) (12650, 1) (12650, 1)
(12650, 1) (12650, 5) (12650, 1) (12650, 1)
(6050, 1) (6050, 5) (6050, 1) (6050, 1)
(6600, 1) (6600, 5) (6600, 1) (6600, 1)
(6600, 1) (6600, 5) (6600, 1) (6600, 1)
(3850, 1) (3850, 5) (3850, 1) (3850, 1)
(5962, 1) (5962, 5) (5962, 1) (5962, 1)
(5962, 1) (5962, 5) (5962, 1) (5962, 1)
(6600, 1) (6600, 5) (6600, 1) (6600, 1)
(6600, 1) (6600, 5) (6600, 1) (6600, 1)
(8800, 1) (8800, 5) (8800, 1) (8800, 1)
(8800, 1) (8800, 5) (8800, 1) (8800, 1)
(8800, 1) (8800, 5) (8800, 1) (8800, 1)
(8800, 1) (8800, 5) (8800, 1) (8800, 1)
(8800, 1) (8800, 5) (8800, 1) (8800, 1)
(8800, 1) (8800, 5) (8800, 1) (8800, 1)
(3300, 1) (3300, 5) (3300, 1) (3300, 1)
(7150, 1) (7150, 5) (7150, 1) (7150, 1)
(7150, 1) (7150, 5) (7150, 1) (7150, 1)
(7150, 1) (7150, 5) (7150, 1) (7150, 1)
(5775, 1) (5775, 5) (5775, 1) (5775, 1)
(6600, 1) (6600, 5) (6600, 1) (6600, 1)
(5225, 1) (5225, 5) (5225, 1

C:\Users\nicho\AppData\Local\Temp\ipykernel_12468\1641646770.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  full_df=np.array(full_df)


In [89]:
df

,EVENT,TEAM_ID,PLAYER_ID,LOC_X,LOC_Y,LOC_Z,GAME_CLOCK,SHOT_CLOCK
0,1.0,-1.0,-1.0,23.4254,45.12734,3.64299,711.26,11.99
1,1.0,1610612761.0,2449.0,19.08811,13.91147,0.0,711.26,11.99
2,1.0,1610612761.0,201960.0,10.11935,13.54703,0.0,711.26,11.99
3,1.0,1610612761.0,200768.0,20.81838,44.51006,0.0,711.26,11.99
4,1.0,1610612761.0,201942.0,14.16535,38.7957,0.0,711.26,11.99
...,...,...,...,...,...,...,...,...
1869654,451,1610612766.0,101107.0,8.02743,45.14262,0.0,0.02,None
1869655,451,1610612766.0,201587.0,53.3,24.32213,0.0,0.02,None
1869656,451,1610612766.0,202689.0,23.36024,19.16116,0.0,0.02,None
1869657,451,1610612766.0,203469.0,44.55981,23.17866,0.0,0.02,None


In [5]:

df = pd.read_csv('../data/merged_csv/0021500001.csv')
df=df[df['EVENT']==2.0]

In [6]:
df.index

Int64Index([1650, 1651, 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659,
            ...
            7173, 7174, 7175, 7176, 7177, 7178, 7179, 7180, 7181, 7182],
           dtype='int64', length=5533)